Initial Attempt

In [1]:
import torch
import torch.nn as nn
import numpy as np

In [2]:

class AddNet(nn.Module):
    def __init__(self):
        super(AddNet, self).__init__()
        #TODO: explain simple linear equation facilitated
        self.output = nn.Linear(2, 1)
    
    def forward(self, x):
        return self.output(x)


In [3]:
def normalize(x):
    """Normalizes input values to range [0,1]."""
    return x / 100.0

def denormalize(x):
    """Denormalizes values back to original range."""
    return x * 100.0

In [4]:

def generate_training_data(num_samples):
    x1 = torch.randint(0, 101, (num_samples, 1), dtype=torch.float32)
    x2 = torch.randint(0, 101, (num_samples, 1), dtype=torch.float32)
    inputs = torch.cat((normalize(x1), normalize(x2)), dim=1)
    targets = normalize(x1 + x2)
    return inputs, targets


In [5]:
def train_add_net(model, num_epochs=500, batch_size=32):
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.05)
    
    inputs, targets = generate_training_data(1000)
    
    for epoch in range(num_epochs):
        idx = torch.randperm(inputs.shape[0])[:batch_size]
        batch_inputs = inputs[idx]
        batch_targets = targets[idx]
        
        outputs = model(batch_inputs)
        loss = criterion(outputs, batch_targets)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (epoch + 1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
            print_weights_and_bias(model)


In [6]:

def inference(model, x1, x2):
    model.eval()
    with torch.no_grad():
        x1 = max(0, min(100, int(x1)))
        x2 = max(0, min(100, int(x2)))
        
        x1_norm = normalize(torch.tensor([[float(x1)]], dtype=torch.float32))
        x2_norm = normalize(torch.tensor([[float(x2)]], dtype=torch.float32))
        inputs = torch.cat((x1_norm, x2_norm), dim=1)
        predicted_sum_norm = model(inputs)
        return denormalize(predicted_sum_norm).item()


In [7]:

def test_addition(model, x1, x2):
    x1 = max(0, min(100, int(x1)))
    x2 = max(0, min(100, int(x2)))
    
    predicted_sum = inference(model, x1, x2)
    actual_sum = x1 + x2
    
    percent_error = abs(predicted_sum - actual_sum) / actual_sum * 100
    
    print(f"\nTest Result:")
    print(f"Numbers: {x1} + {x2}")
    print(f"Predicted sum: {predicted_sum:.4f}")
    print(f"Actual sum: {actual_sum}")
    print(f"Absolute Error: {abs(predicted_sum - actual_sum):.4f}")
    print(f"Percent Error: {percent_error:.2f}%")
    
    return predicted_sum

In [8]:
def batch_test(model, num_samples):
    print(f"\nBatch Testing ({num_samples} samples):")
    print("-" * 50)
    
    total_percent_error = 0
    
    for _ in range(num_samples):
        x1 = np.random.randint(0, 101)
        x2 = np.random.randint(0, 101)
        predicted_sum = test_addition(model, x1, x2)
        actual_sum = x1 + x2
        percent_error = abs(predicted_sum - actual_sum) / actual_sum * 100
        total_percent_error += percent_error
    
    average_percent_error = total_percent_error / num_samples
    print(f"\nAverage Percent Error across {num_samples} samples: {average_percent_error:.2f}%")

In [9]:
def print_weights_and_bias(model):
    weights = model.output.weight.data.numpy()
    bias = model.output.bias.data.numpy()
    print(f"Weights: [{weights[0][0]:.4f}, {weights[0][1]:.4f}]")
    print(f"Bias: {bias[0]:.4f}\n")

In [10]:
model = AddNet()
train_add_net(model)
batch_test(model, 10)

Epoch [100/500], Loss: 0.0288
Weights: [0.5236, 0.5851]
Bias: 0.4519

Epoch [200/500], Loss: 0.0015
Weights: [0.8995, 0.9183]
Bias: 0.0981

Epoch [300/500], Loss: 0.0000
Weights: [0.9896, 0.9917]
Bias: 0.0107

Epoch [400/500], Loss: 0.0000
Weights: [0.9993, 0.9995]
Bias: 0.0007

Epoch [500/500], Loss: 0.0000
Weights: [1.0000, 1.0000]
Bias: 0.0000


Batch Testing (10 samples):
--------------------------------------------------

Test Result:
Numbers: 54 + 80
Predicted sum: 133.9996
Actual sum: 134
Absolute Error: 0.0004
Percent Error: 0.00%

Test Result:
Numbers: 74 + 38
Predicted sum: 111.9998
Actual sum: 112
Absolute Error: 0.0002
Percent Error: 0.00%

Test Result:
Numbers: 5 + 39
Predicted sum: 44.0006
Actual sum: 44
Absolute Error: 0.0006
Percent Error: 0.00%

Test Result:
Numbers: 65 + 91
Predicted sum: 155.9993
Actual sum: 156
Absolute Error: 0.0007
Percent Error: 0.00%

Test Result:
Numbers: 80 + 72
Predicted sum: 151.9994
Actual sum: 152
Absolute Error: 0.0006
Percent Error: 0.00

In [11]:
batch_test(model, 25)


Batch Testing (25 samples):
--------------------------------------------------

Test Result:
Numbers: 65 + 32
Predicted sum: 96.9999
Actual sum: 97
Absolute Error: 0.0001
Percent Error: 0.00%

Test Result:
Numbers: 2 + 12
Predicted sum: 14.0009
Actual sum: 14
Absolute Error: 0.0009
Percent Error: 0.01%

Test Result:
Numbers: 23 + 33
Predicted sum: 56.0004
Actual sum: 56
Absolute Error: 0.0004
Percent Error: 0.00%

Test Result:
Numbers: 46 + 5
Predicted sum: 51.0004
Actual sum: 51
Absolute Error: 0.0004
Percent Error: 0.00%

Test Result:
Numbers: 18 + 59
Predicted sum: 77.0002
Actual sum: 77
Absolute Error: 0.0002
Percent Error: 0.00%

Test Result:
Numbers: 41 + 92
Predicted sum: 132.9996
Actual sum: 133
Absolute Error: 0.0004
Percent Error: 0.00%

Test Result:
Numbers: 75 + 91
Predicted sum: 165.9992
Actual sum: 166
Absolute Error: 0.0008
Percent Error: 0.00%

Test Result:
Numbers: 8 + 58
Predicted sum: 66.0003
Actual sum: 66
Absolute Error: 0.0003
Percent Error: 0.00%

Test Result:
N